In [1]:
import tensorflow as tf
import numpy as np
import sys
import os
#from Prototype import Model


#Rework Loc train routine
#USE PLACEHOLDERS WHILE DOING



In [2]:
#Checking for num of batches
path_to_dataset = os.path.join("data","Dataset")
print(path_to_dataset)
total_num_batches = int((len(os.listdir(path_to_dataset))/2)-1)
print("Found {0} total batches".format(total_num_batches))

#Parameters of photo                        
img_size = 256
num_channels = 3

#Hyper parameters for network
num_classes = 2
batch_size = 10
total_steps = 2
learning_rate_classifier = 1e-4
learning_rate_localizer = 1e-4
keep_probability = 0.7
print_nth_step = 1
result_dir = "./results"
"""
Config variable named params, used for forwarding parameters through
whole model
"""
#Params used as config file
params = {"result_dir":result_dir,
          "learning_rate_cl": learning_rate_classifier,
          "learning_rate_loc": learning_rate_localizer,
          "img_size": img_size,
          "num_channels":num_channels,
          "num_classes":num_classes,
          "batch_size":batch_size,
          "total_batches":total_num_batches,
          "total_steps":total_steps,
          "keep_probability":keep_probability,
          "print_nth_step":print_nth_step}

data\Dataset
Found 499 total batches


In [3]:
import tensorflow as tf
import functools
import os
import time
from my_load_data import load_dataset_fn 

def doublewrap(function):
    """
    A decorator of decorator, allowing use of lazy property if no arguments are provided
    """
    @functools.wraps(function)
    def decorator(*args, **kwargs):
        if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
            return function(args[0])
        else:
            return lambda wrapee: function(wrapee, *args, **kwargs)
    return decorator

@doublewrap
def define_scope(function, scope = None, *args, **kwargs):
    """
    Lazy decorator, optimizes code by loading class Model parts only once to memory
    Also its groups tf.Graph, in tensorboard into smaller, more readable parts
    """    
    attribute = '_cache_' + function.__name__
    name = scope or function.__name__
    
    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            #Sorting Graph by Var_scope
            with tf.variable_scope(name, *args, **kwargs):
                setattr(self, attribute, function(self))
                print("Initialized Model.{}".format(name))
        return getattr(self, attribute)
    
    return decorator


class Model(object):
    """
    Model of neural network with all functionalities
    """
    def __init__(self, params, mode=None):
        #Variables
        self.img_size = params["img_size"]
        self.num_channels = params["num_channels"]
        self.num_classes = params["num_classes"]
        self.lr_cl = params["learning_rate_cl"]
        self.lr_loc = params["learning_rate_loc"]
        self.keep_prob = params["keep_probability"]
        self.total_batches = params["total_batches"]
        self.permutation = np.random.permutation(range(params["total_batches"]))
        self.write_step = params["print_nth_step"]
        self.training = True
        self.total_epoch = 0
        self.data_iter = 1                       
        self.global_step_cl = tf.Variable(0, dtype=tf.int32,
               trainable=False, name='global_step')
        self.data = tf.placeholder(dtype= tf.float32, shape=[None,256,256,3])
        self.target_class = tf.placeholder(dtype=tf.float32, shape=[None,1,1])
        self.target_loc = tf.placeholder(dtype = tf.float32, shape=[None,1,1,1,1])
        
        #Functions returning vars or ops
        self.load_data
        self.prediction
        self.classifier
        self.optimize_cl
        self.loss_cl
        self.localizer
        self.optimize_loc
        self.loss_loc
        
        
    def get_global_step(self):
        return self.global_step_cl.eval()
    
    @define_scope    
    def load_data(self):
        """
        Serve random data each iteration
        """
        with tf.name_scope('Input'):
            self.data, self.target_class, self.target_loc = (load_dataset_fn(self.permutation[self.data_iter]))
            print(type(self.data))
            
            
            
    @define_scope
    def prediction(self):
        """
        Main body of neural network, takes data and labels as input,
        returns feature map of photo
        """
        #INPUT LAYER
        input_layer = tf.reshape(self.data,[-1, self.img_size, self.img_size, self.num_channels])
        #1 conv layer
        conv1 = tf.layers.conv2d(inputs = self.data, 
                             filters = 32,
                             kernel_size = 5,
                             strides = 1,
                             padding = "same",
                             activation = tf.nn.relu)
        
        #1 pool layer, img size reduced by 1/4
        pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                        pool_size = 2, 
                                        strides = 2,
                                        padding = "same")

        #2 conv layer
        conv2 = tf.layers.conv2d(inputs = pool1, 
                             filters = 64,
                             kernel_size = 5,
                             strides = 1,
                             padding = "same",
                             activation = tf.nn.relu)

        #2 pool overal image size reduced totaly by factor of 1/16
        pool2 = tf.layers.max_pooling2d(inputs = conv2,
                                        pool_size = 2, 
                                        strides = 2,
                                        padding = "same")


        pool2_flat = tf.reshape(pool2,[-1,(64*64*64)])

        dense = tf.layers.dense(inputs = pool2_flat,
                            units = 512,
                            activation = tf.nn.relu)

        dense2 = tf.layers.dense(inputs = dense,
                             units = 128,
                             activation = tf.nn.relu)
        #Droupout layer
        self.feature_map = tf.layers.dropout(inputs = dense2, 
                                    rate = self.keep_prob, 
                                    training=self.training)

        return self.feature_map
    
        
    ##################### CLASSIFIER ###############################
    @define_scope
    def classifier(self):
        """
        Assigns class to result
        """
        self.logits_cl = tf.layers.dense(inputs = self.prediction,
                                 units = self.num_classes,
                                 activation = tf.nn.relu)
        y_pred = tf.nn.softmax(logits=self.logits_cl)

        self.pred_cl = y_pred
        return self.pred_cl
    
    @define_scope
    def optimize_cl(self):
        """
        Optimizer of network, call after model.classifier to optimize classifier network
        """
        cross_entropy_cl =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.target_class, logits=self.pred_cl)
        self.opt_cl = tf.train.AdamOptimizer(self.lr_cl).minimize(cross_entropy_cl)
        return self.opt_cl
    
    @define_scope
    def loss_cl(self):
        """
        Function returning loss for classifier
        """
        self.loss_cl_val = tf.losses.softmax_cross_entropy(self.target_class, self.classifier)
        return self.loss_cl_val
    
    def accuracy_cl(self):
        """
        Count the number of right predictions in a batch
        """
        with tf.name_scope('accuracy'):
            preds = tf.nn.softmax(self.logits_cl)
            correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(self.target_class, 1))
            self.accuracy_cl_val = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
            return self.accuracy_cl_val
    
    
    ###################### LOCALIZER ################################

    @define_scope
    def localizer(self):
        """
        Function tries to obtain localization of marker on photo
        """
        #Conversion to num_classes output size
        self.logits_loc = tf.layers.dense(inputs = self.prediction,
                                 units = 4,
                                 activation = tf.nn.relu)
        
        # Softmax output of the neural network
        self.pred_loc = tf.nn.softmax(logits=self.logits_loc)
        return self.pred_loc
    
    @define_scope
    def optimize_loc(self):
        """
        Optimizes localizer
        """
        cross_entropy_loc =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.target_loc, logits=self.pred_loc)
        self.opt_loc = tf.train.AdamOptimizer(self.lr_loc).minimize(cross_entropy_loc)
        return  self.opt_loc
    
    @define_scope
    def loss_loc(self):
        """
        Returns loss for localizator
        """
        self.loss_loc_val = tf.losses.mean_squared_error(self.target_loc, self.pred_loc)
        return  self.loss_loc_val
    
    
    def accuracy_loc(self):
        """
        Count the number of right predictions in a batch
        """
        with tf.name_scope('accuracy'):
            preds = tf.nn.softmax(self.logits_loc)
            correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(self.target_class, 1))
            self.accuracy_loc_val = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
            return self.accuracy_loc_val
    
    
    def summary(self):
        """
        Create summaries to write them to TensorBoard
        """
        with tf.name_scope('summaries'):
            with tf.name_scope('Classifier_summary'):
                tf.summary.scalar('loss', self.loss_cl_val)
                tf.summary.scalar('accuracy', self.accuracy_cl_val)
                tf.summary.histogram('histogram_loss', self.loss_cl_val)
                self.summary_op = tf.summary.merge_all()
                
            return self.summary_op
        
        
        
    def build(self):
        """
        Build the computation graph
        """
        self.accuracy_cl()
        self.accuracy_loc()
        self.summary()
        #self.summary_loc()
        
    
    
  ####################### TRAINING_OPS #######################################  
    
    
    def train_cl(self, ckpt_dir, sess, init, saver, writer,step, epoch):
        """
        Train one epoch of classifier
        """
        start_time = time.time()
        sess.run(init)
        self.training = True
        total_loss = 0
        num_batches = 0
        try: 
            #train
            loss, _, summary = sess.run([self.loss_cl, self.optimize_cl, self.summary_op])
            if (step + 1) % self.write_step == 0:
                print('Loss at step {0}: {1}'.format(step, loss))
                self.data_iter+=1
            #save
            step+=1
            writer.add_summary(summary, global_step=step)
            total_loss += loss
            num_batches += 1
        except tf.errors.OutOfRangeError:
            pass   
        saver.save(sess, ckpt_dir, step)
        print('Average loss at epoch {0}: {1}'.format(epoch, total_loss/num_batches))
        print('Took: {0} seconds'.format(time.time() - start_time))
        return step

    

    def train_loc(self, ckpt_dir, sess, init, saver, writer, step, epoch):
        """
        Train one epoch of localizer
        """
        start_time = time.time()
        sess.run(init)
        self.training = True
        total_loss = 0
        num_batches = 0
        try: 
            #train
            loss, _, summary = sess.run([self.loss_loc, self.optimize_loc, self.summary_op])
            if (step + 1) % self.write_step == 0:
                print('Loss at step {0}: {1}'.format(step, loss))
            #save
            writer.add_summary(summary, global_step=step)
            step += 1
            total_loss += loss
            num_batches += 1
        except tf.errors.OutOfRangeError:
            pass   
        saver.save(sess, ckpt_dir, step)
        print('Average loss at epoch {0}: {1}'.format(epoch, total_loss/num_batches))
        print('Took: {0} seconds'.format(time.time() - start_time))
        return step
    
##### Learning subroutine of network

##### Learning subroutine of network

In [4]:
total_epoch = 100
def main(params, mode = None):
    result_dir = params["result_dir"]
    
    #Check for dirs, if not present make them
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
    ckpt_dir=os.path.join(result_dir,"ckpt")
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)
    
    graph = tf.Graph()
    #Get name as default graph
    with graph.as_default():

        print("Starting Session")
        #Assign name to session, assign it's default graph as graph
        with tf.Session(graph=graph) as sess:
            
            #Creating summary writer 
            writer = tf.summary.FileWriter(ckpt_dir, graph=graph)
                
            #Initialization of Model, load all Model functions returning variables
            model = Model(params, mode = None)
            
            #build graph and load summaries
            model.build()
            
            #Assign Initializer
            init = tf.global_variables_initializer()
            
            #Creating save for model session for future saving and restoring model
            saver = tf.train.Saver()
            
            #Loading last checkpoint
            ckpt = tf.train.get_checkpoint_state(result_dir)
            if ckpt and ckpt.model_checkpoint_path:
                #if ckpt found load it and load global step
                saver.restore(sess, ckpt.model_checkpoint_path)
                print("Found checkpoint")
                step = model.get_global_step()   
            else: step = 1
            
                    
            #Training
            print("Starting Training")
           
            for epoch in range(total_epoch):
                step = model.train_cl( ckpt_dir, sess, init, saver, writer,step, epoch)
              
            
        
        
        print("Finnished session")
        #Merge all summaries
        #writer.flush()
        writer.add_graph(graph)
        writer.close()
        print("Closed summary, work finnished")
        



In [5]:
tf.reset_default_graph()
if __name__ == '__main__':
     main(params)

Starting Session
<class 'tensorflow.python.framework.ops.Tensor'>
Initialized Model.load_data
Initialized Model.prediction
Initialized Model.classifier
Initialized Model.optimize_cl
Initialized Model.loss_cl
Initialized Model.localizer
Initialized Model.optimize_loc
Initialized Model.loss_loc
Starting Training
Loss at step 1: 0.7358612418174744
Average loss at epoch 0: 0.7358612418174744
Took: 51.57481598854065 seconds
Loss at step 2: 0.9665965437889099
Average loss at epoch 1: 0.9665965437889099
Took: 38.523951053619385 seconds
Loss at step 3: 0.6357113122940063
Average loss at epoch 2: 0.6357113122940063
Took: 33.98529124259949 seconds
Loss at step 4: 0.7983590960502625
Average loss at epoch 3: 0.7983590960502625
Took: 55.25214624404907 seconds
Loss at step 5: 0.7573413252830505
Average loss at epoch 4: 0.7573413252830505
Took: 76.79344344139099 seconds
Loss at step 6: 0.7358382940292358
Average loss at epoch 5: 0.7358382940292358
Took: 35.85463619232178 seconds
Loss at step 7: 0.835

Loss at step 71: 0.8429938554763794
Average loss at epoch 70: 0.8429938554763794
Took: 36.814685344696045 seconds
Loss at step 72: 0.7648130655288696
Average loss at epoch 71: 0.7648130655288696
Took: 53.71611452102661 seconds
Loss at step 73: 0.6837564706802368
Average loss at epoch 72: 0.6837564706802368
Took: 38.665282249450684 seconds
Loss at step 74: 0.8391774296760559
Average loss at epoch 73: 0.8391774296760559
Took: 72.30014824867249 seconds
Loss at step 75: 1.0055994987487793
Average loss at epoch 74: 1.0055994987487793
Took: 80.72775363922119 seconds
Loss at step 76: 0.700438380241394
Average loss at epoch 75: 0.700438380241394
Took: 43.12389850616455 seconds
Loss at step 77: 0.7263808250427246
Average loss at epoch 76: 0.7263808250427246
Took: 34.915477991104126 seconds
Loss at step 78: 0.850790798664093
Average loss at epoch 77: 0.850790798664093
Took: 34.094271659851074 seconds
Loss at step 79: 0.8986762166023254
Average loss at epoch 78: 0.8986762166023254
Took: 36.401894

In [ ]:
###Paste into conda prompt
#   tensorboard --logdir="results/"
#   tensorboard --logdir="pythondata/MITP Project/results/"

"""  
    def summary_loc(self):
        """
        
        """
        with tf.name_scope('summaries'):
            with tf.name_scope('Localizer_summary'):
                tf.summary.scalar('loss', self.loss_loc_val)
                tf.summary.scalar('accuracy', self.accuracy_loc_val)
                tf.summary.histogram('histogram_loss', self.loss_loc_val)
                self.summary_op_loc = tf.summary.merge_all()
        return self.summary_op_loc
"""
        
        
        